In [7]:
# import libraries
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup as bs
import re as re
import time
import pandas as pd
import os

In [22]:
# The location of the web driver in your system, a username, and a password to log in with
## https://www.linkedin.com/
## first get the chrome webdriver on your local: https://bit.ly/3FHqvF7
## my webdriver path: /Users/satoshiido/Documents/chromedriver_mac_arm64/chromedriver
PATH = input("Enter the Webdriver path: ")
## input your linkedin account username or email address
USERNAME = input("Enter the username: ")
## input your linkedin account password
PASSWORD = input("Enter the password: ")
print(PATH)
print(USERNAME)
print(PASSWORD)

/Users/satoshiido/Documents/chromedriver_mac_arm64/chromedriver
idsts2670@gmail.com
Ss01182389!


In [23]:
# web documents of WebDriver for Chrome: https://chromedriver.chromium.org/getting-started
# how to include the ChromeDriver location in Mac OS System PATH: https://bit.ly/3lvBnz3
# when you get error: "“chromedriver” cannot be opened because the developer cannot be verified": https://bit.ly/3lqukb1
driver = webdriver.Chrome(PATH)
driver.get("https://www.linkedin.com/uas/login")
time.sleep(3)

# tell the driver to login with the credentials provided above
email = driver.find_element(By.ID, 'username')
email.send_keys(USERNAME)
password = driver.find_element(By.ID, 'password')
password.send_keys(PASSWORD)
time.sleep(3)
password.send_keys(Keys.RETURN)
# driver.quit()

/var/folders/j0/tkdlg4n54w74ht8tklyh79640000gn/T/ipykernel_8776/1811885428.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)


# LinkedIn connections

In [24]:
page = input("Enter the Company or User Linkedin URL: ")
# if its company
company_name = page[33:-1]
# if its user
user_name = page[28:-1]

In [26]:
driver.get(page + 'recent-activity/')

In [28]:
# for company profile
# driver.get(page + 'posts/')
# for personal profile
driver.get(page + 'recent-activity/')
SCROLL_PAUSE_TIME = 1.5
# Get scroll height
last_height = driver.execute_script("return document.body.scrollHeight")


while True:
    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

In [ ]:
def Scrape_func(a,b,c):
    name = a[28:-1]
    page = a
    time.sleep(10)

    driver.get(page + 'detail/recent-activity/shares/')  
    start=time.time()
    lastHeight = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)
        newHeight = driver.execute_script("return document.body.scrollHeight")
        if newHeight == lastHeight:
            break
        lastHeight = newHeight
        end=time.time()
        if round(end-start)>20:
            break

    company_page = driver.page_source   

    linkedin_soup = bs(company_page.encode("utf-8"), "html")
    linkedin_soup.prettify()
    containers = linkedin_soup.findAll("div",{"class":"occludable-update ember-view"})
    print("Fetching data from account: "+ name)
    iterations = 0
    nos = int(input("Enter number of posts: "))
    for container in containers:

        try:
            text_box = container.find("div",{"class":"feed-shared-update-v2__description-wrapper ember-view"})
            text = text_box.find("span",{"dir":"ltr"})
            b.append(text.text.strip())
            c.append(name)
            iterations += 1
            print(iterations)
            
            if(iterations==nos):
                break

        except:
            pass 

n = int(input("Enter the number of entries: "))
for i in range(n):
    post_links.append(input("Enter the link: "))
for j in range(n):
    Scrape_func(post_links[j],post_texts,post_names)

        
driver.quit()

In [ ]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
s=Service(ChromeDriverManager().install())


[WDM] - Downloading: 100%|██████████| 8.82M/8.82M [00:15<00:00, 610kB/s] 


In [3]:
import json
import requests
from dataclasses import dataclass, asdict

@dataclass
class Company():
    name: str
    rank: int
    revenues: float
    revchange: float
    profits: float
    assets: float
    prftchange: float
    employees: int
    rankchange: int
    hqcountry: str
    newcommer: str
    rankdrop: str
    rankgain: str
    sector: str
    industry: str
    hqcity: str
    hqstate: str
    profitable: str
    ceowoman: str
    jobgrowth: str
    fortune500: str
    title: str

    @classmethod
    def from_json(cls, jstring):
        c = {}
        for i in jstring:
            c[i['key'].replace('-', '_').lower()] = \
                    i['value'].replace('&#038;', '&').replace('&#8217;', "'")

        #print(f"Rank: {c['rank']:4}  Name: {c['name']}")
        #print(json.dumps(c, indent=4))
        return cls(c['name'], int(c['rank']), float(c['revenues']), 
                   float(c['revchange']), float(c['profits']), float(c['assets']), 
                   c['prftchange'], int(c['employees']), c['rankchange'], 
                   c['hqcountry'], c['newcomer'], c['rankdrop'], c['rankgain'],
                   c['sector'], c['industry'], c['hqcity'], c['hqstate'],
                   c['profitable'], c['ceowoman'], c['jobgrowth'],
                   c['fortune500_y_n'], c['title'])


def fortune500_list():
    url = 'https://content.fortune.com/wp-json/irving/v1/data/franchise-search-results?list_id=2666483'
    r = requests.get(url)
    #r.encoding = 'ISO-8859-1'
    json_data = json.loads(r.text)[1]['items']
    companies = []
    for comp in json_data:
        companies.append(Company.from_json(comp['fields']))
    return companies


f500 = fortune500_list()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)